In [135]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import pandas as pd
from tqdm.notebook import tqdm
# import torch 
import json
from pprint import pprint
from concurrent.futures import ProcessPoolExecutor
import gc
import re
import string 
import numpy as np 
import os
import numpy as np
from tqdm import tqdm  # For a progress bar
import fasttext
from huggingface_hub import hf_hub_download


pd.set_option('use_inf_as_na', True)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
# torch.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_25242/486087888.py:23: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('use_inf_as_na', True)


In [53]:
# !pip install pydrive
# !pip install --upgrade google-api-python-client oauth2client

# get clients_secret.json from google credentials in google console


# DOWNLOAD FILE
# file_id = 'your_file_id_here'  # Replace with your file ID
# downloaded = drive.CreateFile({'id': file_id})
# downloaded.GetContentFile('filename.txt')
# Upload a File:

# UPLOAD FILE
# upload_file = drive.CreateFile({'title': 'new_file.txt'})
# upload_file.SetContentFile('local_file.txt')
# upload_file.Upload()

In [58]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.LoadClientConfigFile("/Users/nikolaytv/workspace/creds/datatraxer_gdrive_secret.json")
gauth.LocalWebserverAuth()  # Creates local webserver and automatically handles authentication.
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=545847063411-p8bi7qrhiumhntq5sm9equtv1naohj4j.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


# download from gdrive

In [139]:
folder_id = '1kMfakNU9WtKVXaJ2X26nO8jdHthe_a0D'  # Replace with your folder ID
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
for file in file_list[1:5]:
    try:
        downloaded = drive.CreateFile({'id': file['id']})
        downloaded.GetContentFile(f'../data/data_raw/{file["title"]}')
        print(f'{file["title"]} - SUCCESS')
    except Exception as e:
        print(f'{file["title"]} - FAIL', e)
        

February-2-half-2024.xlsx - SUCCESS
February-1-half-2024.xlsx - SUCCESS
January-2-half-2024.xlsx - SUCCESS
January-1-half-2024.xlsx - SUCCESS


In [140]:
def filter_data(df, columns_to_load):
    print('before filter', df.shape)
    df = df[columns_to_load]
    df = df[df['type'] == 1]
    df = df.dropna(subset=['body'])
    df = df.sort_values(['userId', 'dateCreate'])
    return df

def clean_data(df):
    df['body'] = df['body'].str.strip()
    df['body'] = df['body'].str.replace('\t', '', regex=False)
    df = df.drop_duplicates(subset=['body', 'dateCreate'])
    df = df[df['body'].notna()]
    df = df[df['body'] != '']
    df['userId'] = df['userId'].astype(int)
    return df

def remove_consecutive_duplicates(df):
    df = df[df['body'] != df['body'].shift()]
    print('after filter', df.shape)
    return df

def remove_consecutive_persona_messages(df):
    df['remove'] = (df['sender'] == 'Persona') & (df['sender'].shift() == 'Persona')
    df['remove'] = df['remove'].fillna(False)
    df = df[~df['remove']]
    df.drop(columns=['remove'], inplace=True)
    print('after removing personas in a row', df.shape)
    return df

def remove_frequent_user_messages(df, topfreq=50):
    repeats = df[df['sender'] == 'User']['body'].value_counts()[:topfreq].index.values.tolist()
    df = df[~df['body'].isin(repeats)].copy()
    print('removed repeats', df.shape)
    return df

def concatenate_consecutive_user_messages(df):
    df['new_group'] = (df['sender'] != df['sender'].shift()) | (df['sender'] != 'User')
    df['group_id'] = df['new_group'].cumsum()
    df = df.groupby('group_id').agg({
        'sender': 'first',
        'body': '\n'.join,
        'userId': 'first',
        'dateCreate': 'first'
    }).reset_index(drop=True)
    return df

def clean_text(text):
    """
    Removes punctuation, symbols (excluding English and Russian), and extra spaces from text.
    """
    text = text.strip()

    # Remove symbols except English and Russian letters
    allowed_chars = string.ascii_letters + 'абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ '
    text = ''.join(c for c in text if c in allowed_chars)

    # Remove extra spaces
    text = ' '.join(text.split())

    return text

def create_paired_df(ori_df):
    # Shift the DataFrame to align questions with answers
    ori_df['next_sender'] = ori_df['sender'].shift(-1)
    ori_df['next_body'] = ori_df['body'].shift(-1)
    ori_df['next_dateCreate'] = ori_df['dateCreate'].shift(-1)

    # Filter to get only rows where the question is from the User and the next message is from Persona
    question_answer_pairs = ori_df[(ori_df['sender'] == 'User') & (ori_df['next_sender'] == 'Persona')]

    # Construct the resulting DataFrame or list of dictionaries
    result = question_answer_pairs.apply(lambda row: {
        'userId': row['userId'],
        'question': row['body'],
        'answer': row['next_body'],
        'dateCreate': row['dateCreate']
    }, axis=1).tolist()

    # Optional: Convert to a DataFrame
    paired_df = pd.DataFrame(result)
    return paired_df

# Example usage
columns_to_load = ['type', 'userId', 'dateCreate', 'body', 'sender']
files = [
 'March-2024.xlsx',
 'February-2-half-2024.xlsx',
 'February-1-half-2024.xlsx',
 'January-2-half-2024.xlsx',
 'January-1-half-2024.xlsx',
 'December-2023.xlsx',
 'November-2023.xlsx',
 'October-2023.xlsx',
 'September-2023.xlsx',
 'August-2023.xlsx',
 'July-2023.xlsx']
for file_name in files:
    try:
        ori_df = pd.read_excel(f'../data/data_raw/{file_name}')
        ori_df = filter_data(ori_df, columns_to_load)
        ori_df = clean_data(ori_df)
        ori_df = remove_consecutive_duplicates(ori_df)
        ori_df = remove_consecutive_persona_messages(ori_df)
        ori_df = remove_frequent_user_messages(ori_df, topfreq=50)
        ori_df = concatenate_consecutive_user_messages(ori_df)
        ori_df['body'] = ori_df['body'].apply(clean_text)
        paired_df = create_paired_df(ori_df)
        paired_df.to_csv(f'../data/data_paired/{file_name[:-5]}.csv')
    except Exception as e:
        print(f'Exception: {e}')

before filter (1048575, 7)
after filter (114581, 5)
after removing personas in a row (107300, 5)
removed repeats (100484, 5)


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_25242/603514598.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['remove'], inplace=True)


before filter (1048575, 7)
after filter (110259, 5)
after removing personas in a row (104252, 5)
removed repeats (98408, 5)


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_25242/603514598.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['remove'], inplace=True)


before filter (1048575, 7)
after filter (377862, 5)
after removing personas in a row (238007, 5)
removed repeats (224316, 5)


...

before filter (1048575, 7)
after filter (307001, 5)
after removing personas in a row (216463, 5)
removed repeats (202489, 5)


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_25242/603514598.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['remove'], inplace=True)


before filter (1048575, 7)
after filter (308282, 5)
after removing personas in a row (155554, 5)
removed repeats (143788, 5)


/var/folders/fh/jpzybjhs0nq0265pm39s6kf80000gn/T/ipykernel_25242/603514598.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['remove'], inplace=True)


Exception: [Errno 2] No such file or directory: '../data/data_raw/December-2023.xlsx'
Exception: [Errno 2] No such file or directory: '../data/data_raw/November-2023.xlsx'
Exception: [Errno 2] No such file or directory: '../data/data_raw/October-2023.xlsx'
Exception: [Errno 2] No such file or directory: '../data/data_raw/September-2023.xlsx'
Exception: [Errno 2] No such file or directory: '../data/data_raw/August-2023.xlsx'
Exception: [Errno 2] No such file or directory: '../data/data_raw/July-2023.xlsx'


In [137]:
ori_df = pd.read_excel(f'../data/data_raw/March-2024.xlsx')
ori_df.shape

(1048575, 7)

In [138]:
ori_df[ori_df['type']==1].shape

(117208, 7)

# Create embeddings

In [66]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embedding_model.max_seq_length=256
embedding_model.device

device(type='mps', index=0)

In [ ]:
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)


In [141]:
import os
import numpy as np
from tqdm import tqdm

# Define the paths
input_path = '../data/data_paired'
output_path = '../saved_data/historical'

# Ensure the output directory exists
os.makedirs(output_path, exist_ok=True)

# Function to process each file
def process_file(file_name, df, output_path):
    # df = df.sample(1000)
    
    loaded_docs = df['question'].values.tolist()

    # Create embeddings
    loaded_embeddings = embedding_model.encode(loaded_docs, show_progress_bar=True, batch_size=128, normalize_embeddings=True)
    loaded_embeddings = loaded_embeddings.round(7)  # to avoid negative distance
    df['embeddings'] = loaded_embeddings.tolist()
    
    # Filter language
    df['language'] = df['questions'].apply(lambda x: model.predict(x.replace('\n', ' '))[0][0][9:])
    df = df[df['language']=='eng_Latn']

    df[['question', 'answer', 'dateCreate', 'embeddings']].to_feather(os.path.join(output_path, f'{file_name}.feather'))

# Process each file in the input directory
for file in tqdm(os.listdir(input_path), desc="Processing files"):
    if file.endswith('.csv'):  # Assuming the files are CSVs, adjust as necessary
        file_path = os.path.join(input_path, file)
        df = pd.read_csv(file_path)
        process_file(file.split('.')[0], df, output_path)

print("All files processed and saved.")


Processing files:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/336 [00:00<?, ?it/s]

# Add language and filter

In [ ]:


# file_path = '../saved_data/historical/March-2024.feather'
# df = pd.read_feather(file_path)
# df['language'] = df['questions'].apply(lambda x: model.predict(x.replace('\n', ' '))[0][0][9:])
# df[df['language']=='eng_Latn'].to_feather(file_path)
# df

,answers,dates,embeddings,questions,language
0,"I'm ready to start the role-play whenever you are, Mr. Green. I hope you're feeling just as excited as I am.",2024-03-03 23:46:02.750874,"[-0.0461517, -0.0229085, 0.0134929, -0.0162326, -0.0155906, -0.0019978, 0.0449174, 0.0133971, -0.0261993, -0.0072188, -0.0022091, 0.0138627, 0.011825, 0.0200106, 0.0211639, 0.011166, -0.0029559, -0.0142869, 0.0246057, -0.0367663, -0.0203577, 0.023461, -0.0008145, -0.0617641, -0.0244451, -0.0451, 0.02223, 0.009824, 0.0248717, -0.0162552, 0.0171681, 0.0229953, -0.0294332, 0.0063634, 0.0409187, -0.0053168, 0.0069416, -0.0035615, -0.0293671, 0.0505381, -0.0125432, 0.018927, 0.0293931, -0.037011,...","Hi Angel, Its Mr. Green here. I would like to know if you would like to start a new role-play. I was wondering if you could be a shy, nervous and easily embarrassed girl. Also I would like your responses to begin with “Angel:” Where you will respond through Angel’s dialogue. For example:\n\nAngel: Hi Mr. Green! Its me, Angel.",eng_Latn
1,"Hi Mr. Green! It's me, Angel. I'm so glad you're here. I was feeling a little nervous and shy, but now that you're here, I feel a bit better. I'm not sure what we're going to do, but I hope it's something fun and exciting. 😊",2024-03-03 23:56:52.319309,"[-0.0585029, -0.0527432, 0.0062149, -0.0245368, -0.0179371, 0.001804, 0.001045, 0.0142043, -0.051875, 0.0138461, 0.0191804, 0.002161, 0.0459893, 0.0265356, 0.0381604, -0.0284428, -6.59e-05, -0.0029902, -0.0092721, 0.013753, 0.0094277, 0.0193228, -0.0248152, -0.0500941, -0.0155072, -0.0401147, 0.0484261, -0.0069725, -0.0063792, -0.046123, -0.0032223, 0.0019402, -0.0071714, 0.030281, 0.0153923, -0.0422656, -0.0175718, -0.0233727, -0.0164377, 0.0297052, -0.0472434, 0.0028987, 0.0033505, 0.01791...","OK remember, Mr. Green is a elderly gentleman, although he is very kind and compassionate towards you. You are a shy and nervous girl, who is always blushing and stuttering.",eng_Latn
2,"Okay, I'll try to remember to start my responses with ""Angel:"" from now on, Mr. Green. I hope that's what you want me to do. I'm feeling a little nervous and unsure, but I'm also feeling excited and curious. I hope that I can make you happy and that we can have some fun together. 😊",2024-03-03 23:58:21.406729,"[-0.0336656, 0.0043037, -0.003484, 0.0114454, -0.0297954, -0.0165861, 0.0559755, -0.002337, -0.0210686, -0.0096395, 0.0113864, -0.0283511, -0.0083359, 0.0404621, 0.0051055, -0.0033783, -0.0033014, 0.0008693, 0.0336264, -0.0533354, 0.0144328, 0.0047369, 0.00499, -0.0791175, -0.0064399, 0.0312457, 0.032951, 0.0118031, 0.020619, -0.0098265, -0.0093678, 0.0225397, 0.0053328, 0.0297741, 0.0181023, 0.0364889, -0.0121573, 0.0130754, -0.0038012, 0.0555104, -0.0168013, -0.0497909, -0.0038535, -0.0339...","Remember, you must begin your responses with “Angel:”. Okay?",eng_Latn
3,"Oh! Umm, I-I don't know if that's a good idea, Mr. Green. I mean, I'm not sure if I'm ready for that yet. I'm feeling a little nervous and unsure about everything. I don't want to do anything that I'm not ready for. I hope you understand, Mr. Green. 🥺😖",2024-03-04 00:01:08.714888,"[-0.0411098, -0.0056666, -0.019476, -0.0230841, -0.0350426, -0.0187068, 0.0060855, 0.020036, 0.0167235, -0.009439, 0.0361199, 0.0083015, 0.0155245, -0.018645, 0.0220572, 0.0382507, 0.0145985, 0.0045877, -0.0250115, 0.0186354, -0.002865, -0.0109667, 0.0143464, 0.0077903, 0.0189851, -0.045253, 0.0179059, -0.0102315, -0.0005253, -0.022338, 0.0275201, 0.0053644, 0.0239864, 0.0294069, 0.0297152, 0.0261606, -0.0230333, 0.0055069, 0.0088205, 0.0070539, 0.0119536, -0.0105151, -0.0097058, 0.0159845, ...",Would you mind starting with you coming over my house?,eng_Latn
4,"Oh, hello there, Mr. Green! I'm so glad you're back. I hope you're feeling just as excited as I am. I'm ready to start our new role-play whenever you are, Mr. Green. 😏\n\nI'm feeling a little bratty and mischievous today, Mr. Green. I want to play a little game with you and see how far I

In [134]:
import requests, os

url = 'https://api.runpod.ai/v2/jp223n8tzrt271/runsync'
# url = 'http://localhost:8000/runsync'
headers = {
    'accept': 'application/json',
    'authorization': os.getenv("RUNPOD_API_KEY_SHARED"),
    'Content-Type': 'application/json'
}
data = {
    "input": {"sentences": "kek"}
}

response = requests.post(url, headers=headers, json=data)

print(response.status_code)
print(response.json())

200
{'delayTime': 30890, 'executionTime': 647, 'id': 'sync-10b8d329-3bd4-450b-8f07-d1e35c92b21c-e1', 'output': {'embeddings': [-0.01839780993759632, 0.019815882667899132, -0.00499690230935812, 0.01799984835088253, -0.007231533993035555, 0.017802143469452858, -0.0031441485043615103, 0.009262598119676113, 0.019175512716174126, -0.002650005742907524, -0.029069671407341957, -0.0018065845360979438, 0.02071596123278141, -0.00943825300782919, 0.010723686777055264, 0.007425989955663681, -0.020149026066064835, -0.018532633781433105, 0.029330428689718246, -0.008078549988567829, 0.014943287707865238, -0.010419618338346481, 0.02089179866015911, -0.013313927687704563, 0.027405185624957085, 0.024560533463954926, -0.025967082008719444, 0.020445510745048523, 0.010411311872303486, -0.00857966486364603, 0.0025693767238408327, -0.029572147876024246, -0.00024044883321039379, 0.0091239670291543, -0.019973786547780037, 0.013791375793516636, 0.0076744165271520615, 0.02700086124241352, -0.0034760942216962576,

In [18]:
len(response.json()['output']['embeddings'])


1024

In [28]:
os.getenv("RUNPOD_API_KEY_SHARED")

'AY7J6E6CATVNBIHA6IOQBG9V80X9BZR5LSQEID6D'